In [0]:
%sh sudo apt-get update && apt-get install -y gdal-bin libeccodes-dev libnetcdf-dev libhdf5-dev

In [0]:
%pip install numpy==1.24.4 eccodes==1.7.1 xarray==2024.9.0 cf_xarray==0.10.0 cfgrib==0.9.14.1 netcdf4==1.7.2
%pip install --no-cache-dir --force-reinstall cfgrib==0.9.14.1 numpy==1.24.4

In [0]:
import xarray as xr

In [0]:
# "u10"     = "u10"     # 10m u-component of wind
# "v10"     = "v10"     # 10m v-component of wind
# "u100"    = "u100"    # 100m u-component of wind
# "v100"    = "v100"    # 100m v-component of wind
# "t2m"     = "tas"     # 2m temperature
# "mx2t"    = "tasmax"  # Maximum 2m temperature
# "mn2t"    = "tasmin"  # Minimum 2m temperature
# "msl"     = "mslp"    # Mean sea level pressure
# "ssrd"    = "itssrd"  # Surface solar radiation downwards
# "pr"      = "tp"      # Total precipitation

In [0]:
path = "/Volumes/cdp_prd_sandbox_catalog_01/weather_bronze/bronze/era5/single_hourly/2025/05/02/"
target = "/Volumes/cdp_prd_sandbox_catalog_01/weather_silver/silver/era5_test/v6"
files = dbutils.fs.ls(path)
filters = ["total_precipitation", "surface_solar_radiation_downwards", 
           "maximum_2m_temperature", "minimum_2m_temperature"]
files = [file for file in files if any([f in file.name for f in filters])]
print(len(files))
print(files)

In [0]:
import sys
import os
import re

if (config_path := re.split(r"notebooks", os.getcwd())[0] + "config") not in sys.path:
    sys.path.append(config_path)

if (package_path := re.split(r"notebooks", os.getcwd())[0] + "weather_package") not in sys.path:
    sys.path.append(package_path)

In [0]:
from grib2netcdf.era5_converter import ERA5Grib2NetCDFConverter

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
for file in files:
    try:
        print(f"Processing: {file.name}")
        file_path = file.path.replace("dbfs:", "")
        category = "single_hourly"
        converter = ERA5Grib2NetCDFConverter()
        categories = converter.get_categories()
        if category in categories:
            converter.convert(file_path, target, category)
    except Exception as ex:
            print(f"{ex}\n\n")